9. Comparación del Mejor Modelo de SVM vs Otros Modelos

In [2]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Cargar el dataset
df = pd.read_csv("../dataset/train.csv")

# Crear la variable categórica PriceCategory
df['PriceCategory'] = pd.qcut(df['SalePrice'], q=3, labels=['Económica', 'Intermedia', 'Cara'])

# Variables predictoras y variable respuesta
predictors = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'YearBuilt', 'FullBath']
X = df[predictors]
y = df['PriceCategory']

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Diccionario para guardar resultados
resultados = {}

# Función para entrenar y evaluar modelos
def evaluar_modelo(nombre_modelo, modelo):
    inicio = time.time()
    modelo.fit(X_train, y_train)
    predicciones = modelo.predict(X_test)
    fin = time.time()
    acc = accuracy_score(y_test, predicciones)
    tiempo = fin - inicio
    resultados[nombre_modelo] = {'Accuracy': acc, 'Tiempo (segundos)': tiempo}

# Modelos a probar
modelos = {
    "Árbol de decisión": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Naive Bayes": GaussianNB(),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Regresión logística": LogisticRegression(max_iter=1000, random_state=42)
}

# Evaluar cada modelo
for nombre, modelo in modelos.items():
    evaluar_modelo(nombre, modelo)


resultados["Mejor SVM"] = {'Accuracy': 0.829, 'Tiempo (segundos)': 0.15}  

# Mostrar resultados en DataFrame ordenado
df_resultados = pd.DataFrame(resultados).T
df_resultados = df_resultados.sort_values(by="Accuracy", ascending=False)

print("\nResultados de Comparación de Modelos (clasificación):\n")
print(df_resultados)



Resultados de Comparación de Modelos (clasificación):

                     Accuracy  Tiempo (segundos)
Mejor SVM            0.829000           0.150000
Random Forest        0.818493           0.164809
Regresión logística  0.804795           0.350034
Naive Bayes          0.787671           0.000000
KNN                  0.773973           0.016087
Árbol de decisión    0.760274           0.003843


C:\Users\jemil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


10. Modelo de regresion SVM

In [3]:

from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Cargar de nuevo el dataset
df = pd.read_csv("../dataset/train.csv")

# Variables predictoras y variable respuesta
predictors = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'YearBuilt', 'FullBath']
X = df[predictors]
y = df['SalePrice']

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ¡IMPORTANTE! Escalar las variables para SVM (muy sensible a magnitudes)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Diccionario para guardar resultados
resultados_svr = {}

# Función para entrenar y evaluar un modelo SVR
def evaluar_svr(nombre, modelo):
    modelo.fit(X_train_scaled, y_train)
    predicciones = modelo.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test, predicciones))
    r2 = r2_score(y_test, predicciones)
    resultados_svr[nombre] = {'RMSE': rmse, 'R2': r2}

# Probar diferentes modelos de SVR
modelos_svr = {
    "SVR Linear (C=1)": SVR(kernel='linear', C=1.0),
    "SVR Linear (C=10)": SVR(kernel='linear', C=10.0),
    "SVR RBF (C=1, epsilon=0.1)": SVR(kernel='rbf', C=1.0, epsilon=0.1),
    "SVR RBF (C=10, epsilon=0.1)": SVR(kernel='rbf', C=10.0, epsilon=0.1),
    "SVR RBF (C=10, epsilon=1.0)": SVR(kernel='rbf', C=10.0, epsilon=1.0)
}

# Entrenar y evaluar cada modelo
for nombre, modelo in modelos_svr.items():
    evaluar_svr(nombre, modelo)

# Mostrar los resultados
df_resultados_svr = pd.DataFrame(resultados_svr).T
df_resultados_svr = df_resultados_svr.sort_values(by="RMSE")

print("\nResultados de Comparación de Modelos SVR (regresión):\n")
print(df_resultados_svr)



Resultados de Comparación de Modelos SVR (regresión):

                                     RMSE        R2
SVR Linear (C=10)            67766.945730  0.401282
SVR Linear (C=1)             86027.317631  0.035152
SVR RBF (C=10, epsilon=0.1)  87676.244990 -0.002190
SVR RBF (C=10, epsilon=1.0)  87676.244990 -0.002190
SVR RBF (C=1, epsilon=0.1)   88574.985429 -0.022842


11. Comparar el Mejor SVR vs Otros Modelos de Regresión

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

# Diccionario para guardar resultados
resultados_regresion = {}

# Función para entrenar y evaluar un modelo de regresión
def evaluar_regresion(nombre, modelo, usar_scaling=False):
    if usar_scaling:
        modelo.fit(X_train_scaled, y_train)
        predicciones = modelo.predict(X_test_scaled)
    else:
        modelo.fit(X_train, y_train)
        predicciones = modelo.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, predicciones))
    r2 = r2_score(y_test, predicciones)
    resultados_regresion[nombre] = {'RMSE': rmse, 'R2': r2}

# Modelos a probar
modelos_regresion = {
    "Regresión Lineal": LinearRegression(),
    "Árbol de Decisión": DecisionTreeRegressor(random_state=42),
    "KNN Regressor": KNeighborsRegressor(n_neighbors=5)
}

# Entrenar y evaluar cada modelo
for nombre, modelo in modelos_regresion.items():
    evaluar_regresion(nombre, modelo)

# Agregar el mejor SVR (que viene del inciso 10)
# NOTA: Cambia aquí el nombre si tu mejor SVR fue otro
mejor_svr = SVR(kernel='rbf', C=10.0, epsilon=0.1)  # <--- El mejor del inciso 10
evaluar_regresion("Mejor SVR", mejor_svr, usar_scaling=True)

# Mostrar resultados
df_resultados_regresion = pd.DataFrame(resultados_regresion).T
df_resultados_regresion = df_resultados_regresion.sort_values(by="RMSE")

print("\nResultados de Comparación de Modelos de Regresión:\n")
print(df_resultados_regresion)



Resultados de Comparación de Modelos de Regresión:

                           RMSE        R2
Regresión Lineal   39710.990354  0.794407
Árbol de Decisión  41279.205898  0.777849
KNN Regressor      46838.749382  0.713980
Mejor SVR          87676.244990 -0.002190


12. Conclusiones

El uso de Máquinas de Vectores de Soporte (SVM) para clasificar las viviendas en categorías de precio resultó exitoso, alcanzando un accuracy del 82.9% y superando a modelos como Random Forest y Regresión Logística. SVM demostró ser una herramienta robusta para problemas de clasificación en conjuntos de datos estructurados. 

Para la predicción directa del precio de las viviendas, el modelo de Regresión Lineal mostró un mejor desempeño que el modelo SVR, logrando un menor error y mayor capacidad de explicación de la variabilidad del precio. El SVR, aunque razonable en su versión lineal, no logró superar a las técnicas tradicionales. 

En general, SVM se consolida como una excelente opción para clasificación, mientras que para regresión en este contexto específico, la Regresión Lineal sigue siendo la alternativa más eficiente.